In [33]:
import pandas as pd
import datetime

In [35]:
youthpay = pd.read_csv("All Months-Days Worked Data.csv", parse_dates=['Start Date', 'End Date'])
youth = pd.read_csv("Combined Youth Data-All Youth - Raw Data  .csv", parse_dates=['Start Date', 'End Date'])

In [36]:
holidays = pd.to_datetime([
    "2023-03-27", "2023-03-28", "2023-03-29", "2023-03-30", "2023-03-31",
    "2023-04-01", "2023-04-02", "2023-04-03", "2023-04-04", "2023-04-05",
    "2023-04-06", "2023-04-07", "2023-04-08", "2023-04-09", "2023-04-10",
    "2023-04-11", "2023-06-26", "2023-06-27", "2023-06-28", "2023-06-29",
    "2023-06-30", "2023-07-01", "2023-07-02", "2023-07-03", "2023-07-04",
    "2023-07-05", "2023-07-06", "2023-07-07", "2023-07-08", "2023-07-09",
    "2023-07-10", "2023-07-11", "2023-07-12", "2023-07-13", "2023-07-14",
    "2023-07-15", "2023-07-16", "2023-07-17"
])


In [37]:
def workdays_in_month(year, month, holidays, start_date=None, end_date=None):
    """
    Calculate the number of workdays in a given month and year, excluding weekends and given holidays.
    Optionally consider a start and end date.
    """
    # Define the start and end of the month
    month_start = pd.Timestamp(year, month, 1)
    if month == 12:
        month_end = pd.Timestamp(year + 1, 1, 1) - pd.Timedelta(days=1)
    else:
        month_end = pd.Timestamp(year, month + 1, 1) - pd.Timedelta(days=1)
    
    # Adjust for provided start and end dates
    if start_date and start_date > month_start:
        month_start = start_date
    if end_date and end_date < month_end:
        month_end = end_date

    # Generate date range for the month
    date_range = pd.date_range(month_start, month_end)
    
    # Filter out weekends and holidays
    workdays = date_range[~date_range.isin(holidays)]
    workdays = workdays[workdays.dayofweek < 5]
    
    return len(workdays)



In [38]:
months = ['March', 'April', 'May', 'June', 'July']
year = 2023

for index, row in youthpay.iterrows():
    for month in months:
        month_num = pd.Timestamp(f"{year}-{month}-01").month
        
        # Extract the youth's start and end dates
        start_date = row['Start Date'] if month_num >= row['Start Date'].month else None
        end_date = row['End Date'] if month_num <= row['End Date'].month else None
        
        youthpay.at[index, f"{month} - Workdays"] = workdays_in_month(year, month_num, holidays, start_date, end_date)


In [39]:
youthpay[['Employee ID', 'Start Date', 'End Date', 'March - Days','March - Workdays']].sort_values('March - Days', ascending=False)

,Employee ID,Start Date,End Date,March - Days,March - Workdays
125,509,2021-05-01,NaT,20.0,18.0
162,593,2022-08-01,NaT,20.0,18.0
154,573,2022-01-04,NaT,20.0,18.0
168,661,2022-06-01,NaT,20.0,18.0
127,514,2022-08-01,NaT,20.0,18.0
...,...,...,...,...,...
572,307,2022-01-04,2022-11-30,NaN,0.0
573,306,2022-01-04,2022-11-30,NaN,0.0
574,233,2023-05-30,NaT,NaN,18.0
577,201,2021-02-11,2023-02-05,NaN,18.0


In [17]:
hired_after_22 = youth[youth['Start Date'].between("2022/01/01", "2023/04/01")]
hired_after_22[['Employee ID', 'Start Date', 'End Date', 'Months Active', 'Job Title', 'Reason for Leaving']].sort_values(by="Start Date")

,Employee ID,Start Date,End Date,Months Active,Job Title,Reason for Leaving
135,444,2022-01-01,2023-04-26,15,Literacy Coach,Found a Job
136,445,2022-01-01,NaT,19,Literacy Coach,NaN
219,545,2022-01-02,2022-12-13,11,Librarian,Found a Job
254,580,2022-01-02,2022-11-30,10,Literacy Coach,Did Not Return The Following Year
230,556,2022-01-02,2022-12-13,11,Literacy Coach,Found a Job
...,...,...,...,...,...,...
332,717,2023-03-24,NaT,5,1000 Stories Youth,NaN
476,866,2023-03-27,2023-03-27,0,Literacy Coach,Found a Job
477,867,2023-03-27,NaT,5,Literacy Coach,NaN
478,868,2023-03-27,NaT,5,Literacy Coach,NaN


In [40]:
# Group by 'Job Title' and calculate average months active and turnover rate
grouped_data = youth.groupby('Job Title').apply(lambda x: pd.Series({
    'Average Months Active': x['Months Active'].mean(),
    'Turnover Rate': (len(x[x['End Date'].dt.date != pd.Timestamp.today().date()]) / len(x)) * 100
})).reset_index()

grouped_data


,Job Title,Average Months Active,Turnover Rate
0,1000 Stories Youth,1.243243,100.0
1,EduTech Coach,11.615385,100.0
2,Librarian,22.894737,100.0
3,Literacy Coach,9.877193,100.0
4,Sport & Arts Coach,3.666667,100.0
5,Yeboneer,7.022831,100.0


In [26]:
# Extract the month from the 'End Date' column
youth['End Month'] = youth['End Date'].dt.month

# Count the number of departures for each month
departure_counts = youth[youth['End Date'].dt.date != pd.Timestamp.today().date()]['End Month'].value_counts().sort_index()

departure_counts


End Month
1.0       3
2.0      16
3.0      23
4.0      16
5.0      33
6.0      10
7.0      17
8.0      14
9.0       1
10.0      2
11.0    120
12.0     48
Name: count, dtype: int64

In [31]:
# Filter youths that started in 2023
youth_2023 = youth[youth['Start Date'].dt.year == 2023]


# Filter out youths with 'Did Not Start' in the 'Reason for Leaving' column
youth_2023_excluded = youth_2023[youth_2023['Reason for Leaving'] != 'Did Not Start']

# Calculate turnover rate for youths that started in 2023 excluding 'Did Not Start'
total_youth_2023_excluded = len(youth_2023_excluded)
youth_2023_excluded_left = len(youth_2023_excluded[youth_2023_excluded['End Date'].dt.date != pd.Timestamp.today().date()])

turnover_rate_2023_excluded = (youth_2023_excluded_left / total_youth_2023_excluded) * 100

turnover_rate_2023_excluded


100.0

In [41]:


# Convert Start Date and End Date columns to datetime format
youth['Start Date'] = pd.to_datetime(youth['Start Date'])
youth['End Date'].fillna(pd.Timestamp.today(), inplace=True)  # Fill missing End Dates with today's date
youth['End Date'] = pd.to_datetime(youth['End Date'])

# Calculate number of months active for each youth using years and months
youth['Months Active'] = (youth['End Date'].dt.year - youth['Start Date'].dt.year) * 12 + \
                        (youth['End Date'].dt.month - youth['Start Date'].dt.month)

# Filter youths that started in 2023 and exclude those with 'Did Not Start' in the 'Reason for Leaving' column
youth_2023 = youth[youth['Start Date'].dt.year == 2023]
youth_2023_excluded = youth_2023[youth_2023['Reason for Leaving'] != 'Did Not Start']

# Calculate turnover rate for youths that started in 2023 excluding 'Did Not Start'
total_youth_2023_excluded = len(youth_2023_excluded)
youth_2023_excluded_left = len(youth_2023_excluded[youth_2023_excluded['End Date'].dt.date != pd.Timestamp.today().date()])
turnover_rate_2023_excluded = (youth_2023_excluded_left / total_youth_2023_excluded) * 100

print(f"Turnover rate for youths who started in 2023: {turnover_rate_2023_excluded:.2f}%")

# Breakdown by Job Title
grouped_data = youth_2023_excluded.groupby('Job Title').apply(lambda x: pd.Series({
    'Average Months Active': x['Months Active'].mean(),
    'Turnover Rate': (len(x[x['End Date'].dt.date != pd.Timestamp.today().date()]) / len(x)) * 100
})).reset_index()

print(grouped_data)


Turnover rate for youths who started in 2023: 20.44%
            Job Title  Average Months Active  Turnover Rate
0  1000 Stories Youth               2.722222      22.222222
1       EduTech Coach               3.500000      33.333333
2      Literacy Coach               3.826087      21.739130
3  Sport & Arts Coach               4.666667       0.000000
4            Yeboneer               4.366972      18.348624


In [42]:
youth_22 = youth[youth['Start Date'].between("2022/01/01", "2022/11/01")]


In [44]:
youth_22.groupby('Job Title')['Months Active'].mean()

Job Title
EduTech Coach     10.000000
Librarian         15.166667
Literacy Coach    11.942308
Yeboneer           9.971963
Name: Months Active, dtype: float64

In [52]:
youth_22['Employment Status'].value_counts()

Employment Status
Inactive    180
Active       41
Name: count, dtype: int64

In [58]:
youth_22['Reason for Leaving'] = youth['Reason for Leaving'].fillna("Still Active")

youth_22.groupby(['Employment Status', 'Job Title'])['Reason for Leaving'].value_counts()

/var/folders/pl/dqfqzz7579g7h98m6pygrdyw0000gn/T/ipykernel_41842/2598177223.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  youth_22['Reason for Leaving'] = youth['Reason for Leaving'].fillna("Still Active")


Employment Status  Job Title       Reason for Leaving                
Active             EduTech Coach   Still Active                           1
                   Librarian       Still Active                           4
                   Literacy Coach  Still Active                          36
Inactive           EduTech Coach   Found a Job                            2
                                   Did Not Return The Following Year      1
                   Librarian       Found a Job                            2
                   Literacy Coach  Found a Job                           31
                                   Personal (Chose Not To Specify)       10
                                   Did Not Return The Following Year     10
                                   Went to University / College           8
                                   Dismissed                              4
                                   Passed Away                            2
                  